# Tutorial-02: Cleaning up and More Operations

We start as usual by loading up some libraries we will need.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# The Food Establisment Inspection Data
df = pd.read_csv("https://data.boston.gov/dataset/03693648-2c62-4a2c-a4ec-48de2ee14e18/resource/4582bec6-2b4f-4f9e-bc55-cbaa73117f4c/download/tmpdbqq_szy.csv")

In [ ]:
df.head(5)

In [ ]:
df['businessname'].value_counts()

We need to convert Dunkin' to Dunkin and this is done with .replace({"this":"with that"})

In [ ]:
df['businessname'] = df['businessname'].replace({"Dunkin' Donuts":"Dunkin Donuts"})

In [ ]:
# Did it work?
df['businessname'].value_counts()

We could do multiple replace statements in a single line of code. 

In [ ]:
df['businessname'] = df['businessname'].replace({"Dunkin' Donuts":"Dunkin Donuts", "Mcdonalds":"McDonald's"})

In [ ]:
df['businessname'].value_counts()

Let us see what we can find for establishments in Lexington

In [ ]:
df_lex = df[ df["city"] == "Lexington" ]

In [ ]:
df_lex

In [ ]:
df_bos = df[ df['city'] == "Boston" ]

In [ ]:
df_bos

In [ ]:
df_bos.head(5)

What establishments have the most violations in Boston?

In [ ]:
df_bos['businessname'].value_counts()

What legal owner seems to have the most violations? 

In [ ]:
df['legalowner'].value_counts()

Now it might a good idea to dig into this McCoy dude.

In [ ]:
df_mccoy = df[ df['legalowner'] == "MCCOY RICHARD" ]

In [ ]:
df_mccoy['businessname'].value_counts()

What do each of the first four businesses get most dinged for?

In [ ]:
df_mccoy.groupby(['businessname'])['comments'].value_counts()

## Back to the Boston Marathon

In [ ]:
bm15 = pd.read_csv("data/marathon_results_2015.csv")
bm16 = pd.read_csv("data/marathon_results_2016.csv")
bm17 = pd.read_csv("data/marathon_results_2017.csv")

bm15 = bm15.drop(columns = 'Unnamed: 0')
bm17 = bm17.drop(columns = 'Unnamed: 0')

bm_df = pd.concat([bm15, bm16, bm17])

In [ ]:
bm_df.head(5)

### Now we ask, using just the 2017 data: 

(1) What country had the most runners?  
(2) Does this vary between Male and Female runners?

In [ ]:
bm17['Country'].value_counts()

In [ ]:
# It would be good to eliminate the USA rows since we know most runners will be domestic.

bm17[bm17.Country != "USA"]['Country'].value_counts()

In [ ]:
# Now we look at Men and Women but before we do that we rename that column since M/F will cause trouble. 

bm17.rename(columns = {"M/F":"MaleFemale"}, inplace = True)

# `inplace = True` ensures that the column name is overwritten in bm17 otherwise it is just a temporary name change. 

In [ ]:
bm17[bm17.MaleFemale == "F"]['Country'].value_counts()

In [ ]:
bm17[bm17.MaleFemale == "M"]['Country'].value_counts()

Now let us see the average age, average finishing time, and so on. We will use the Mean and the Median. 

In [ ]:
list(bm17.columns)

In [ ]:
bm17.rename(columns = {"Official Time":"FinishTime", "Proj Time":"ProjectedTime"}, inplace = True)

bm17.describe()

Hmm, why is it not giving us anything more than these four columns? Because the other columns are not stored as numbers. We will have to convert them before we can calculate a mean, etc. But converting wil also involve oe extra step ... they are reporting times in hours, minutes, and seconds but it will have to be either in seconds or minutes. 

In [ ]:
bm17['FinishTime'] = pd.to_timedelta(bm17['FinishTime']).astype('timedelta64[s]').astype(int).reset_index()

In [ ]:
# The command that follows will show us how each column is stored. 
# Anything listed as an `object` implies it is stored as something other than a number.

bm17.dtypes

In [ ]:
# Now we check the means again with `.describe()`

bm17.describe()

In [ ]:
# What about averages by MaleFemale?

bm17.groupby('MaleFemale').describe()

In [ ]:
# Maybe seconds was a bad idea. How about if we convert it to Minutes? To hours?

bm17['FinishTimeM'] = bm17['FinishTime']/60 
bm17['FinishTimeH'] = bm17['FinishTime']/3600 

In [ ]:
bm17.groupby('MaleFemale').describe()

In [ ]:
import seaborn as sns
sns.set_theme()

In [ ]:
sns.displot(bm17, x = "Age")

In [ ]:
# What if we split it out by Age?

sns.displot(bm17, x = "Age", hue = "MaleFemale", col = "MaleFemale", multiple = "dodge")

Maybe we organzie this better, by collapsing runners into age categories. 

In [ ]:
sns.displot(bm17, x = "Age", hue = "MaleFemale", col = "MaleFemale", multiple = "dodge", bins = 10)

In [ ]:
sns.displot(bm17, x = "Age", hue = "MaleFemale", col = "MaleFemale", multiple = "dodge", bins = 5)

### Creating a grouped version of Age

In [ ]:
# Say we want the groups to be <= 20, 21-30, 31-40, 41-50, 51-60, 61-70, 70+

mybins = [0, 21, 31, 41, 51, 61, 71, 100]

mylabels = ["<= 20", "21-30", "31-40", "41-50", "51-60", "61-70", "71+"]

bm17['AgeGrouped'] = pd.cut(bm17['Age'], bins = mybins, labels = mylabels, ordered = True)

`IMPORTANT:` You need one less label that you have distinct values in the binds. Compare `mybins` to `mylabels`

In [ ]:
# I am using `sort = False` so that we see the number of runners in each age group in ascending order of the age
# groups. If we do not add this the result will be sorted in descending order of the frequencies 

bm17['AgeGrouped'].value_counts(sort = False)

Now we can mess around with the data more by asking, what is mean and median finishing time by age-group of male versus female runners

In [ ]:
bm17.groupby(['MaleFemale', 'AgeGrouped'])['FinishTimeH'].mean().round(2)

In [ ]:
# Means can distort the picture if we have skewed distributions so how about the median instead?
bm17.groupby(['MaleFemale', 'AgeGrouped'])['FinishTimeH'].median().round(2)

Some interesting stuff here in that The fastest average time is in the 21-30 group of both Men and Women, no matter what measure of "average" one takes. 

In [ ]:
# Now we pull out some more statistics to see which group has the highest variance 

bm17.groupby(['MaleFemale', 'AgeGrouped'])['FinishTimeH'].aggregate(['median', 'mean', 'std', 'var']).round(2)

Surprise, surprise, it is the 21-30 group for both Males and Females. The winners are in this age-bracket so we should have expectd this from the outset. Least variability is in the 71+ age-bracket. 

### How many runners ran in only one of the races but not the others? How many ran in all three?

In [ ]:
# We are creating a new column called `nrow` that will look at unique combinations of Name, M/F, City, and Country 
# and count how many rows do we see per person. The maximum should be 3, indicating these individuals ran in all three
# marathons

bm_df['nrow'] = bm_df.groupby(['Name', 'M/F', 'City', 'Country']).cumcount()+1

In [ ]:
# Now the frequency table ...

bm_df['nrow'].value_counts()

In [ ]:
# Were men more likely to run in all three or were women more likely to do so? Answer turns out to be Men. 

bm_df.groupby(['M/F'])['nrow'].value_counts()

## Et voila, le fin!!